# Minimal OpenSCM-Runner example with FaIR

Requires fair >= 1.6.0c3 and < 2.0.0

In [ ]:
# NBVAL_IGNORE_OUTPUT
import openscm_runner

In [ ]:
# NBVAL_IGNORE_OUTPUT
print(openscm_runner.__version__)

In [ ]:
import os.path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyam
from fair.forward import fair_scm
from fair.ancil import natural, cmip6_solar, cmip6_volcanic
from scmdata import ScmRun
from tqdm import tqdm_notebook

from openscm_runner.run import run
from openscm_runner.adapters import FAIR
from openscm_runner.utils import calculate_quantiles

In [ ]:
fair = FAIR()

In [ ]:
# NBVAL_IGNORE_OUTPUT
fair.get_version()

In [ ]:
# NBVAL_IGNORE_OUTPUT
df_fair = ScmRun(
    os.path.join(
        "..", "tests", "test-data", "rcmip_scen_ssp_world_emissions.csv"
    ),
    lowercase_cols=True
)

df_fair.head(30)

In [ ]:
# NBVAL_IGNORE_OUTPUT
x = run(
    climate_models_cfgs={
        "FAIR": [
            {},  # passing an empty list of an empty dict will run with defaults
            {"q": np.array([0.3, 0.45]), "r0": 30.0, "lambda_global": 0.9},
            {"q": np.array([0.35, 0.4]), "r0": 25.0, "lambda_global": 1.1}, 
        ], 
    },
    scenarios=df_fair,
    output_variables=(
        "Surface Temperature", 
        "Atmospheric Concentrations|CO2",
        "Effective Radiative Forcing",
        "Effective Radiative Forcing|CO2",
        "Effective Radiative Forcing|Aerosols",
        "Effective Radiative Forcing|Aerosols|Direct Effect|BC",
        "Effective Radiative Forcing|Aerosols|Direct Effect|OC",
        "Effective Radiative Forcing|Aerosols|Direct Effect|Sulfur",
        "Effective Radiative Forcing|Aerosols|Direct Effect",
        "Effective Radiative Forcing|Aerosols|Indirect Effect",
    ),
)

Note in the plots below that 'model' is the IAM that produced the scenario. In all cases, the climate model is FaIR.

In [ ]:
# NBVAL_IGNORE_OUTPUT
x.get_unique_meta("climate_model", no_duplicates=True)

In [ ]:
# NBVAL_IGNORE_OUTPUT
ax = plt.figure(figsize=(12, 7)).add_subplot(111)
x.filter(variable="Surface Temperature").lineplot(
    hue="scenario", style="model", ax=ax, time_axis="year"
)
ax.axhline(1.1)
ax.axvline(2018)

In [ ]:
# NBVAL_IGNORE_OUTPUT
ax = plt.figure(figsize=(12, 7)).add_subplot(111)
x.filter(variable="Atmospheric Concentrations|CO2").lineplot(
    hue="scenario", style="model", ax=ax
)

In [ ]:
# NBVAL_IGNORE_OUTPUT
ax = plt.figure(figsize=(12, 7)).add_subplot(111)
x.filter(variable="Effective Radiative Forcing*", scenario="ssp245").lineplot(
    hue="variable", ax=ax
)

In [ ]:
x.filter(variable="Surface Temperature", year=2100, scenario=["ssp126"]).values.max()

In [ ]:
x.filter(variable="Surface Temperature", year=2100, scenario=["ssp126"]).values.min()

In [ ]:
x.filter(variable="Surface Temperature", year=2100, scenario=["ssp370"]).values.max()

In [ ]:
x.filter(variable="Surface Temperature", year=2100, scenario=["ssp370"]).values.min()

In [ ]:
quantiles = calculate_quantiles(x, [0.05, 0.17, 0.5, 0.83, 0.95])

In [ ]:
quantiles.filter(variable="Surface Temperature", year=2100, scenario=["ssp126"], quantile=0.05).values[0][0]

In [ ]:
quantiles.filter(variable="Surface Temperature", year=2100, scenario=["ssp126"], quantile=0.95).values[0][0]

In [ ]:
quantiles.filter(variable="Surface Temperature", year=2100, scenario=["ssp370"], quantile=0.05).values[0][0]

In [ ]:
quantiles.filter(variable="Surface Temperature", year=2100, scenario=["ssp370"], quantile=0.95).values[0][0]